In [5]:
import pandas as pd
import bs4
import requests
import numpy as np
import time
from time import sleep
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [6]:
#Pandas params
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

def infer_games():
    seasonlist = []
    gamenums = []
    gamelist = []
    season = 1996
    while season < 2020:
        seasonlist.append(season)
        season +=1
    era = [season for season in seasonlist]

    for season in era:
        start = str(season)[2:4]+'00001'
        end = str(season)[2:4]+'01301'
        numberlist = [number for number in range(int(start),int(end))]
        
        for n in numberlist:
                gamenums.append(n)

    for number in gamenums:
        game = '002'+(str(number))
        gamelist.append(game)
    return gamelist

pd.DataFrame(infer_games())

In [11]:
def scrape_pbp():
    seasonlist = []
    gamenums = []
    gamelist = []
    season = 1996
    while season < 2020:
        seasonlist.append(season)
        season +=1
    era = [season for season in seasonlist]

    for season in era:
        #Although leading zeroes will drop off when converting to int, the '2' is necessary in order for game ids fromm 2000-01 to be properly formatted
        start = '002'+str(season)[2:4]+'00001'
        end = '002'+str(season)[2:4]+'01301'
        numberlist = [number for number in range(int(start),int(end))]
        
        for n in numberlist:
                gamenums.append(n)

    for number in gamenums:
        game = '00'+(str(number))
        gamelist.append(game)
    schedule = gamelist
        
    for game in schedule: 
        time.sleep(np.random.randint(0,2))
        try: 
            headers = {
                            'Host': 'stats.nba.com',
                            'Connection': 'keep-alive',
                            'Accept': 'application/json, text/plain, */*',
                            'x-nba-stats-token': 'true',
                            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
                            'x-nba-stats-origin': 'stats',
                            'Referer': f'https://stats.nba.com/game/{game}/playbyplay/',
                            'Accept-Encoding': 'gzip, deflate, br',
                            'Accept-Language': 'en-US,en;q=0.9'}
            
            url = f"https://stats.nba.com/stats/playbyplayv2?EndPeriod=10&EndRange=55800&GameID={game}&RangeType=2&Season={season}&SeasonType=Regular+Season&StartPeriod=1&StartRange=0"                
            session = requests.Session()
            retry = Retry(connect=3, backoff_factor=0.5)
            adapter = HTTPAdapter(max_retries=retry)
            session.mount('http://', adapter)
            session.mount('https://', adapter)
            r = session.get(url, headers=headers).json()
                    
            df = pd.DataFrame(r['resultSets'][0]['rowSet'], columns = r['resultSets'][0]['headers'])
            df.to_csv(f"C:\\Users\gsteele\pbp\{game}_pbp.csv", index=False)
                    
        except LookupError:
            continue

In [ ]:
scrape_pbp()

In [ ]:
import glob

path = r'C:\Users\gsteele\boxes\boxes'
all_files = glob.glob(path + "/*.csv")

csv_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    csv_list.append(df)

frame = pd.concat(csv_list, axis=0, ignore_index=True)

In [ ]:
frame.to_csv("all_boxes.csv", index=False, encoding='utf-8-sig')